## Welcome to ACME eCommerce LLC 

This is your first day on the job. Everyone of our empoyees is its own special flower...

Now make us money

ACME eCommerce is in the business of selling things to people. Your first task is to tell me how many people we had on the site.

## Lets create our first kafka topic
Open a bash session inside the kafka image and create a new topic 
```
docker ps | grep kafka
# Copy kafka hash
docker exec -it DOCKER_CONTAINER_ID /bin/bash

# if you only have one kafka running the follwing will work:
docker exec -it `docker ps | grep kafka | awk '{print $1}'` /bin/bash
# from inside the docker image
$KAFKA_HOME/bin/kafka-topics.sh --create --topic visit_log \
    --zookeeper zk --partitions 2 --replication-factor 1

```

Now, lets setup some visit data first. Please open the notebook [03 - Producer](/notebooks/03%20-%20Producer.ipynb) in a new tab and run through all the code there. Then return here.

In [53]:
import json
import pykafka
 
docker_ip = '192.168.99.100'
client = pykafka.KafkaClient(hosts="{}:9092".format(docker_ip))

In [20]:
client.topics

{'enriched_visit_log': <pykafka.topic.Topic at 0x10aa1b050 (name=enriched_visit_log)>,
 'visit_log': <pykafka.topic.Topic at 0x10aa1b110 (name=visit_log)>}

In [39]:
topic = client.topics['visit_log'] 

In [40]:
topic.partitions

{0: <pykafka.partition.Partition at 0x10aa1b150 (id=0)>,
 1: <pykafka.partition.Partition at 0x10aa1b190 (id=1)>}

In [41]:
c = topic.get_simple_consumer()

In [42]:
consumer = topic.get_simple_consumer()

In [43]:
message = consumer.consume(block=False)

In [44]:
message.offset

0

In [45]:
# WE HAVE OUR FIRST MESSAGE.
message.value

'1442226072.8 [main] INFO Index.html user:9799'

We can loop through all the messages with this code. But be careful if you have lots of messages in your topic!

In [57]:
%%time
count = 0

while True:
    message = consumer.consume(block=False)
    if not message:
        break
    # YOUR CODE HERE
    count += 1
print "We counsumed: {} messages".format(count)

We counsumed: 0 messages
CPU times: user 37 µs, sys: 6 µs, total: 43 µs
Wall time: 43.2 µs


So your first coding task. 
Given this stream answers your bosses first question. How many unique users have visited the site?

In [ ]:
consumer = topic.get_simple_consumer()

In [ ]:
%%time

while True:
    message = consumer.consume(block=False)
    if not message:
        break
    # YOUR CODE HERE

# Enrichment

The most common tasks when processing streams is enriching the data.

Take for example, how many users do we have per country?

We need to enrich the current data because it has no idea of country.

In [34]:
# lets build a lookup table in in-memory
import random
countries = ['USA', 'USA', 'USA', 'UK', 'UK', 'Canada', 'Germany']
user_country_lookup = {str(i): random.choice(countries) for i in range(10000)}

In [ ]:
consumer = topic.get_simple_consumer()

In [ ]:
from collections import Counter
counter = Counter()

In [ ]:
count = 0

while True:
    message = consumer.consume(block=False)
    if not message:
        break
    # YOUR CODE HERE
    user_id = message.value.split(':')[1]
    if user_id in user_country_lookup:
        counter[user_country_lookup[user_id]] += 1
    else:
        counter['Unknown'] += 1
    count += 1
print "We counsumed: {} messages".format(count)

In [ ]:
counter

Now ACME eCommerce LLC is an agile, out-side-the-box, Big Data driven, industry disrupting innovation machine. So your not the only one who wants this data...

All your wonderful work on data enrichment has gotten around. People are clamoring to get their hands on it.

First lets create a new kafka topic.

Like we did before open a terminal. Open a bash session inside the kafka image and 

```
# if you only have one kafka running the follwing will work:
docker exec -it `docker ps | grep kafka | awk '{print $1}'` /bin/bash

$KAFKA_HOME/bin/kafka-topics.sh --create --topic enriched_visit_log \
    --zookeeper zk --partitions 2 --replication-factor 1

```

In [17]:
client.topics

{'enriched_visit_log': <pykafka.topic.Topic at 0x1074cbd90 (name=enriched_visit_log)>,
 'visit_log': <pykafka.topic.Topic at 0x1074cbe50 (name=visit_log)>}

In [525]:
enriched_visit_topic = client.topics['enriched_visit_log1']
enriched_visit_producer = enriched_visit_topic.get_producer()

In [68]:
consumer = topic.get_simple_consumer()

In [526]:
def enrich_message(message):
    """
    Takes a raw visit message and 
    returns a dict of attributes about that message.
    """
    message_data = {}
    # WRITE YOUR CODE HERE
    user_id = message.value.split(':')[1]
    message_data['user_id'] = user_id
    
    return message_data

In [527]:
enrich_message(message)

IndexError: list index out of range

In [528]:
# now process the topic, saving the results to another topic
while True:
    message = consumer.consume(block=False)
    if not message:
        break
    # YOUR CODE HERE
    enriched_message = enrich_message(message)
    enriched_visit_producer.produce([json.dumps(enriched_message)])

KeyboardInterrupt: 

In [531]:
enriched_visit_consumer = enriched_visit_topic.get_simple_consumer()

In [532]:
message = enriched_visit_consumer.consume(block=False)

In [533]:
message.value

'{"user_id": "9799"}'